In [183]:
import requests
import pandas as pd
import joblib
import json


In [75]:
header_list = ['datetime', 'road_number', 'km', 'direction', 'all_units', 'inflow_units',
               'outflow_unit', 'samecell_units', 'avg_speed', 'max_speed', 'avg_traveltime', 'max_traveltime']

In [76]:
num_cols = ['all_units', 'inflow_units',
            'portion_speed', 'max_speed',
            'avg_traveltime', 'max_traveltime']

In [181]:
df_km127 = pd.read_csv("../dataset/latlon_km127.csv")

In [80]:
def map_traffic_with_latlon(df):
    df['lat'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values) > 0 else 0, axis=1)
    df['lon'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values) > 0 else 0, axis=1)
    return df

In [74]:
road = 2
direction = "in"
sus_cluster = 1

In [81]:
gnb_model = joblib.load(f'../Models/GaussianNBModelRoad{road}{direction}.joblib')

In [82]:
road_data = pd.read_csv("http://analytics2.dlt.transcodeglobal.com/cell_data/current_celldata.csv",names=header_list)

In [ ]:
dfm = pd.read_csv(f'../dataset/road{road}-{direction}-mffs.csv')

In [84]:
road_data = road_data[(road_data['road_number'] == road)]

In [85]:
road_data['mffs'] = road_data['km'].map(dfm.set_index('km')['mffs'])

In [86]:
road_data = map_traffic_with_latlon(road_data)

In [89]:
road2in_data = road_data[(road_data['road_number']==road) & (road_data['direction']=='in')]

In [90]:
road2in_data['portion_speed'] = road2in_data['avg_speed']/road2in_data['mffs']

C:\Users\Tananett\AppData\Local\Temp/ipykernel_6980/1312273665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  road2in_data['portion_speed'] = road2in_data['avg_speed']/road2in_data['mffs']


In [92]:
x_predict = road2in_data[num_cols]

In [ ]:
y_predict = gnb_model.predict(x_predict)

In [ ]:
road2in_data['cluster'] = y_predict

In [96]:
y_predict_proba = gnb_model.predict_proba(x_predict)

In [ ]:
problist = []
for i in y_predict_proba :
    problist.append(round(i[sus_cluster],3))


In [ ]:
road2in_data['prob'] = problist

In [102]:
road2in_data = road2in_data[(road2in_data['cluster']==sus_cluster) & (road2in_data['prob']>=0.8)]

In [173]:
post_data = road2in_data[['road_number','km','direction','inflow_units','outflow_unit','samecell_units','all_units','avg_speed','lat','lon','datetime']]

In [177]:
post_data = post_data.values.tolist()

In [179]:
url = 'https://tad-api-v1.herokuapp.com/api/accident/bulk'
# headers = {'Content-type': 'application/json','Accept': 'application/json'}
result = requests.post(url, json=post_data)

In [180]:
print(result.status_code)
print(result.reason)

200
OK
